In [7]:
pwd

'/Users/Chen/Developer/Repository/Clevo-Emotion-Detection-Service/serverless/dev'

In [83]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

batch_size = 128
# num_classes = 10
epochs = 12


In [84]:

mono=True


def get_class_names(path="Preproc/"):  # class names are subdirectory names in Preproc/ directory
    class_names = os.listdir(path)
    class_names.remove(".DS_Store")
    return class_names

def get_total_files(path="Preproc/",train_percentage=0.8): 
    sum_total = 0
    sum_train = 0
    sum_test = 0
    subdirs = os.listdir(path)
    subdirs.remove(".DS_Store")

    for subdir in subdirs:
        files = os.listdir(path+subdir)
        # files.remove(".DS_Store")
        n_files = len(files)
        sum_total += n_files
        n_train = int(train_percentage*n_files)
        n_test = n_files - n_train
        sum_train += n_train
        sum_test += n_test
    return sum_total, sum_train, sum_test

def get_sample_dimensions(path='Preproc/'):
    classnames = os.listdir(path)
    classnames.remove(".DS_Store")
    classname = classnames[0]
    files = os.listdir(path+classname)
    # files.remove(".DS_Store")
    infilename = files[0]
    audio_path = path + classname + '/' + infilename
    
    print("os.path.exists(audio_path)", os.path.exists(audio_path))
    
    melgram = np.load(audio_path)
    print("   get_sample_dimensions: melgram.shape = ",melgram.shape)
    return melgram.shape
 

def encode_class(class_name, class_names):  # makes a "one-hot" vector for each class name called
    try:
        idx = class_names.index(class_name)
        vec = np.zeros(len(class_names))
        vec[idx] = 1
        return vec
    except ValueError:
        return None

def shuffle_XY_paths(X,Y,paths):   # generates a randomized order, keeping X&Y(&paths) together
    assert (X.shape[0] == Y.shape[0] )
    idx = np.array(range(Y.shape[0]))
    np.random.shuffle(idx)
    newX = np.copy(X)
    newY = np.copy(Y)
    newpaths = paths
    for i in range(len(idx)):
        newX[i] = X[idx[i],:,:]
        newY[i] = Y[idx[i],:]
        newpaths[i] = paths[idx[i]]
    return newX, newY, newpaths


'''
So we make the training & testing datasets here, and we do it separately.
Why not just make one big dataset, shuffle, and then split into train & test?
because we want to make sure statistics in training & testing are as similar as possible
'''
def build_datasets(train_percentage=0.8, preproc=False):
    if (preproc):
        path = "Preproc/"
    else:
        path = "Samples/"

    class_names = get_class_names(path=path)
    print("class_names = ",class_names)

    total_files, total_train, total_test = get_total_files(path=path, train_percentage=train_percentage)
    print("total files = ",total_files)

    nb_classes = len(class_names)

    # pre-allocate memory for speed (old method used np.concatenate, slow)
    mel_dims = get_sample_dimensions(path=path)  # Find out the 'shape' of each data file
    X_train = np.zeros((total_train, mel_dims[1], mel_dims[2], mel_dims[3]))   
    Y_train = np.zeros((total_train, nb_classes))  
    X_test = np.zeros((total_test, mel_dims[1], mel_dims[2], mel_dims[3]))  
    Y_test = np.zeros((total_test, nb_classes))  
    paths_train = []
    paths_test = []

    train_count = 0
    test_count = 0
    for idx, classname in enumerate(class_names):
        this_Y = np.array(encode_class(classname,class_names) )
        this_Y = this_Y[np.newaxis,:]
        class_files = os.listdir(path+classname)
        # class_files.remove(".DS_Store")
        n_files = len(class_files)
        n_load =  n_files
        n_train = int(train_percentage * n_load)
        printevery = 100
        print("")
        for idx2, infilename in enumerate(class_files[0:n_load]):          
            audio_path = path + classname + '/' + infilename
            if (0 == idx2 % printevery):
                print('\r Loading class: {:14s} ({:2d} of {:2d} classes)'.format(classname,idx+1,nb_classes),
                       ", file ",idx2+1," of ",n_load,": ",audio_path,sep="")
            #start = timer()
            if (preproc):
                melgram = np.load(audio_path)
                sr = 8000                
#                 sr = 44100
            else:
              aud, sr = librosa.load(audio_path, mono=mono,sr=None)
              melgram = librosa.logamplitude(librosa.feature.melspectrogram(aud, sr=sr, n_mels=96),ref_power=1.0)[np.newaxis,np.newaxis,:,:]

            melgram = melgram[:,:,:,0:mel_dims[3]]   # just in case files are differnt sizes: clip to first file size
       
            #end = timer()
            #print("time = ",end - start) 
            if (idx2 < n_train):
                # concatenate is SLOW for big datasets; use pre-allocated instead
                #X_train = np.concatenate((X_train, melgram), axis=0)  
                #Y_train = np.concatenate((Y_train, this_Y), axis=0)
                X_train[train_count,:,:] = melgram
                Y_train[train_count,:] = this_Y
                paths_train.append(audio_path)     # list-appending is still fast. (??)
                train_count += 1
            else:
                X_test[test_count,:,:] = melgram
                Y_test[test_count,:] = this_Y
                #X_test = np.concatenate((X_test, melgram), axis=0)
                #Y_test = np.concatenate((Y_test, this_Y), axis=0)
                paths_test.append(audio_path)
                test_count += 1
        print("")

    print("Shuffling order of data...")
    X_train, Y_train, paths_train = shuffle_XY_paths(X_train, Y_train, paths_train)
    X_test, Y_test, paths_test = shuffle_XY_paths(X_test, Y_test, paths_test)

    return X_train, Y_train, paths_train, X_test, Y_test, paths_test, class_names, sr

In [86]:

# get the data
x_train, y_train, paths_train, x_test, y_test, paths_test, class_names, sr = build_datasets(preproc=True)

# # input image dimensions
img_rows, img_cols = 96, 157




class_names =  ['Love, Happiness', 'Sadness, Sorrow']
total files =  46
os.path.exists(audio_path) True
   get_sample_dimensions: melgram.shape =  (1, 1, 96, 157)

 Loading class: Love, Happiness ( 1 of  2 classes), file 1 of 24: Preproc/Love, Happiness/334490_209_091532__3.wav.npy


 Loading class: Sadness, Sorrow ( 2 of  2 classes), file 1 of 22: Preproc/Sadness, Sorrow/334491_208_091642__3.wav.npy

Shuffling order of data...


In [87]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print(y_train.shape)

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)



(36, 96, 157, 1) (36, 2) (10, 96, 157, 1) (10, 2)
x_train shape: (36, 96, 157, 1)
36 train samples
10 test samples
(36, 2)


In [88]:
print("x_train.shape", x_train.shape)
print("y_train.shape", y_train.shape)
print("x_test.shape", x_test.shape)
print("y_test.shape", y_test.shape)
# print("class_names", class_names)
print("input_shape", input_shape)

x_train.shape (36, 96, 157, 1)
y_train.shape (36, 2)
x_test.shape (10, 96, 157, 1)
y_test.shape (10, 2)
input_shape (96, 157, 1)


In [90]:

# plt.figure(figsize=(10,10))
# start = 9;
# plt.subplot(221)
# plt.imshow(x_train[start+0,:,:,0])
# plt.subplot(222)
# plt.imshow(x_train[start+1,:,:,0])
# plt.subplot(223)
# plt.imshow(x_train[start+2,:,:,0])
# plt.subplot(224)
# plt.imshow(x_train[start+3,:,:,0])

In [91]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
#           validation_split = 0.2
          validation_data=(x_test, y_test)
         )
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 36 samples, validate on 10 samples
Epoch 1/12
36/36 [==============================] - 3s - loss: 0.6933 - acc: 0.3889 - val_loss: 0.7000 - val_acc: 0.5000
Epoch 2/12
36/36 [==============================] - 1s - loss: 0.6818 - acc: 0.5278 - val_loss: 0.6807 - val_acc: 0.5000
Epoch 3/12
36/36 [==============================] - 1s - loss: 0.6656 - acc: 0.6111 - val_loss: 0.6646 - val_acc: 0.7000
Epoch 4/12
36/36 [==============================] - 1s - loss: 0.6325 - acc: 0.7500 - val_loss: 0.6521 - val_acc: 0.6000
Epoch 5/12
36/36 [==============================] - 1s - loss: 0.6056 - acc: 0.7222 - val_loss: 0.8414 - val_acc: 0.5000
Epoch 6/12
36/36 [==============================] - 2s - loss: 0.7004 - acc: 0.5556 - val_loss: 0.7364 - val_acc: 0.5000
Epoch 7/12
36/36 [==============================] - 1s - loss: 0.7422 - acc: 0.4722 - val_loss: 0.6730 - val_acc: 0.5000
Epoch 8/12
36/36 [==============================] - 1s - loss: 0.6156 - acc: 0.6111 - val_loss: 0.6505 - val_

In [24]:
# batch_size = 128
# num_classes = 10
# epochs = 12

# # input image dimensions
# img_rows, img_cols = 28, 28

# # the data, shuffled and split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# print(y_train.shape)
# print(y_test.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(60000, 10)
(10000, 10)


In [ ]:
count = 10
cate = 8

for i,y_train_data in enumerate(y_train):
    if y_train_data[cate] == 1:
        count-=1
        x_train[i]
        plt.figure()
        plt.imshow(x_train[i,:,:,0])
    if count == 0:
        break